In [1]:
import datetime
import copy
import xarray as xr
import numpy as np
import pandas as pd
import fsspec
import kerchunk
from kerchunk.grib2 import scan_grib, grib_tree
import gcsfs
import datatree
import pickle


# This could be generalized to any gridded FMRC da/taset but right now it works with NOAA's Grib2 files
import dynamic_zarr_store

axes = [
  pd.Index(
    [
      pd.timedelta_range(start="0 hours", end="6 hours", freq="6h", closed="right", name="6 hour"),
    ],
    name="step"
  ),
  pd.date_range("2023-09-01T06:00", "2023-10T00:00", freq="360min", name="valid_time")
]
axes

[Index([[0 days 06:00:00]], dtype='object', name='step'),
 DatetimeIndex(['2023-09-01 06:00:00', '2023-09-01 12:00:00',
                '2023-09-01 18:00:00', '2023-09-02 00:00:00',
                '2023-09-02 06:00:00', '2023-09-02 12:00:00',
                '2023-09-02 18:00:00', '2023-09-03 00:00:00',
                '2023-09-03 06:00:00', '2023-09-03 12:00:00',
                ...
                '2023-09-28 18:00:00', '2023-09-29 00:00:00',
                '2023-09-29 06:00:00', '2023-09-29 12:00:00',
                '2023-09-29 18:00:00', '2023-09-30 00:00:00',
                '2023-09-30 06:00:00', '2023-09-30 12:00:00',
                '2023-09-30 18:00:00', '2023-10-01 00:00:00'],
               dtype='datetime64[ns]', name='valid_time', length=120, freq='360min')]

In [4]:

mapping = dynamic_zarr_store.build_idx_grib_mapping(
    fs=fsspec.filesystem(""),
    basename="/home/sparrow/Documents/sewaa_data/gfs_demo_data/gfs.t00z.pgrb2.0p25.f006",
)
mapping

Dropping unknown variable in msg# 0. Compare with the grib idx file to help identify it and build an ecCodes local grib definitions file to fix it.
Dropping unknown variable in msg# 0. Compare with the grib idx file to help identify it and build an ecCodes local grib definitions file to fix it.
Dropping unknown variable in msg# 0. Compare with the grib idx file to help identify it and build an ecCodes local grib definitions file to fix it.
Dropping unknown variable in msg# 0. Compare with the grib idx file to help identify it and build an ecCodes local grib definitions file to fix it.
The idx attribute mapping for /home/sparrow/Documents/sewaa_data/gfs_demo_data/gfs.t00z.pgrb2.0p25.f006 is not unique for 4 variables: ['tp', 'tp', 'acpcp', 'acpcp']
The grib hierarchy in /home/sparrow/Documents/sewaa_data/gfs_demo_data/gfs.t00z.pgrb2.0p25.f006 is not unique for 30 variables: ['st', 'soilw', 'soill', 'st', 'soilw', 'soill', 'st', 'soilw', 'soill', 'st', 'soilw', 'soill', nan, nan, nan, 't

,offset_idx,date,attrs,length_idx,idx_uri,grib_uri,indexed_at,grib_crc32,grib_updated_at,idx_crc32,...,stepType,name,level,step,time,valid_time,uri,offset_grib,length_grib,inline_value
idx,,,,,,,,,,,,,,,,,,,,,
1,0,d=2023092800,PRMSL:mean sea level:6 hour fcst:\n,984804,/home/sparrow/Documents/sewaa_data/gfs_demo_da...,/home/sparrow/Documents/sewaa_data/gfs_demo_da...,2024-06-24 15:10:07.208587,None,None,None,...,instant,Pressure reduced to MSL,0.000000e+00,0 days 06:00:00,2023-09-28,2023-09-28 06:00:00,/home/sparrow/Documents/sewaa_data/gfs_demo_da...,0.0,984804.0,None
2,984804,d=2023092800,CLWMR:1 hybrid level:6 hour fcst:\n,85627,/home/sparrow/Documents/sewaa_data/gfs_demo_da...,/home/sparrow/Documents/sewaa_data/gfs_demo_da...,2024-06-24 15:10:07.208587,None,None,None,...,instant,Cloud mixing ratio,1.000000e+00,0 days 06:00:00,2023-09-28,2023-09-28 06:00:00,/home/sparrow/Documents/sewaa_data/gfs_demo_da...,984804.0,85627.0,None
3,1070431,d=2023092800,ICMR:1 hybrid level:6 hour fcst:\n,244445,/home/sparrow/Documents/sewaa_data/gfs_demo_da...,/home/sparrow/Documents/sewaa_data/gfs_demo_da...,2024-06-24 15:10:07.208587,None,None,None,...,instant,Ice water mixing ratio,1.000000e+00,0 days 06:00:00,2023-09-28,2023-09-28 06:00:00,/home/sparrow/Documents/sewaa_data/gfs_demo_da...,1070431.0,244445.0,None
4,1314876,d=2023092800,RWMR:1 hybrid level:6 hour fcst:\n,236628,/home/sparrow/Documents/sewaa_data/gfs_demo_da...,/home/sparrow/Documents/sewaa_data/gfs_demo_da...,2024-06-24 15:10:07.208587,None,None,None,...,instant,Rain mixing ratio,1.000000e+00,0 days 06:00:00,2023-09-28,2023-09-28 06:00:00,/home/sparrow/Documents/sewaa_data/gfs_demo_da...,1314876.0,236628.0,None
5,1551504,d=2023092800,SNMR:1 hybrid level:6 hour fcst:\n,78043,/home/sparrow/Documents/sewaa_data/gfs_demo_da...,/home/sparrow/Documents/sewaa_data/gfs_demo_da...,2024-06-24 15:10:07.208587,None,None,None,...,instant,Snow mixing ratio,1.000000e+00,0 days 06:00:00,2023-09-28,2023-09-28 06:00:00,/home/sparrow/Documents/sewaa_data/gfs_demo_da...,1551504.0,78043.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,535590536,d=2023092800,VGRD:PV=-2e-06 (Km^2/kg/s) surface:6 hour fcst:\n,646273,/home/sparrow/Documents/sewaa_data/gfs_demo_da...,/home/sparrow/Documents/sewaa_data/gfs_demo_da...,2024-06-24 15:10:07.208587,None,None,None,...,instant,V component of wind,2.147486e+09,0 days 06:00:00,2023-09-28,2023-09-28 06:00:00,/home/sparrow/Documents/sewaa_data/gfs_demo_da...,535590536.0,646273.0,None
740,536236809,d=2023092800,TMP:PV=-2e-06 (Km^2/kg/s) surface:6 hour fcst:\n,650363,/home/sparrow/Documents/sewaa_data/gfs_demo_da...,/home/sparrow/Documents/sewaa_data/gfs_demo_da...,2024-06-24 15:10:07.208587,None,None,None,...,instant,Temperature,2.147486e+09,0 days 06:00:00,2023-09-28,2023-09-28 06:00:00,/home/sparrow/Documents/sewaa_data/gfs_demo_da...,536236809.0,650363.0,None
741,536887172,d=2023092800,HGT:PV=-2e-06 (Km^2/kg/s) surface:6 hour fcst:\n,1174640,/home/sparrow/Documents/sewaa_data/gfs_demo_da...,/home/sparrow/Documents/sewaa_data/gfs_demo_da...,2024-06-24 15:10:07.208587,None,None,None,...,instant,Geopotential height,2.147486e+09,0 days 06:00:00,2023-09-28,2023-09-28 06:00:00,/home/sparrow/Documents/sewaa_data/gfs_demo_da...,536887172.0,1174640.0,None


In [5]:
%%time
mapped_index_list = []

deduped_mapping = mapping.loc[~mapping["attrs"].duplicated(keep="first"), :]
for date in pd.date_range("2023-09-01", "2023-09-30"):
  for runtime in range(0,24,6):
    horizon=6
    fname=f"gs://global-forecast-system/gfs.{date.strftime('%Y%m%d')}/{runtime:02}/atmos/gfs.t{runtime:02}z.pgrb2.0p25.f{horizon:03}"

    idxdf = dynamic_zarr_store.parse_grib_idx(
        fs=fsspec.filesystem("gcs"),
        basename=fname
    )

    mapped_index = dynamic_zarr_store.map_from_index(
        pd.Timestamp( date + datetime.timedelta(hours=runtime)),
        deduped_mapping,
        idxdf.loc[~idxdf["attrs"].duplicated(keep="first"), :],
    )
    mapped_index_list.append(mapped_index)

gfs_kind = pd.concat(mapped_index_list)
gfs_kind

CPU times: user 3.63 s, sys: 132 ms, total: 3.76 s
Wall time: 3min


,varname,typeOfLevel,stepType,name,step,level,time,valid_time,uri,offset,length,inline_value,grib_crc32,grib_updated_at,idx_crc32,idx_updated_at,indexed_at
0,prmsl,meanSea,instant,Pressure reduced to MSL,0 days 06:00:00,0.000000e+00,2023-09-01 00:00:00,2023-09-01 06:00:00,gs://global-forecast-system/gfs.20230901/00/at...,0,877361,None,0f2Mww==,2023-09-01 03:36:52.570,ZmkdeQ==,2023-09-01 03:34:44.443,2024-06-24 15:16:33.560532
1,clwmr,hybrid,instant,Cloud mixing ratio,0 days 06:00:00,1.000000e+00,2023-09-01 00:00:00,2023-09-01 06:00:00,gs://global-forecast-system/gfs.20230901/00/at...,877361,90753,None,0f2Mww==,2023-09-01 03:36:52.570,ZmkdeQ==,2023-09-01 03:34:44.443,2024-06-24 15:16:33.560532
2,icmr,hybrid,instant,Ice water mixing ratio,0 days 06:00:00,1.000000e+00,2023-09-01 00:00:00,2023-09-01 06:00:00,gs://global-forecast-system/gfs.20230901/00/at...,968114,228036,None,0f2Mww==,2023-09-01 03:36:52.570,ZmkdeQ==,2023-09-01 03:34:44.443,2024-06-24 15:16:33.560532
3,rwmr,hybrid,instant,Rain mixing ratio,0 days 06:00:00,1.000000e+00,2023-09-01 00:00:00,2023-09-01 06:00:00,gs://global-forecast-system/gfs.20230901/00/at...,1196150,266763,None,0f2Mww==,2023-09-01 03:36:52.570,ZmkdeQ==,2023-09-01 03:34:44.443,2024-06-24 15:16:33.560532
4,snmr,hybrid,instant,Snow mixing ratio,0 days 06:00:00,1.000000e+00,2023-09-01 00:00:00,2023-09-01 06:00:00,gs://global-forecast-system/gfs.20230901/00/at...,1462913,81052,None,0f2Mww==,2023-09-01 03:36:52.570,ZmkdeQ==,2023-09-01 03:34:44.443,2024-06-24 15:16:33.560532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
732,v,potentialVorticity,instant,V component of wind,0 days 06:00:00,2.147486e+09,2023-09-30 18:00:00,2023-10-01 00:00:00,gs://global-forecast-system/gfs.20230930/18/at...,535413574,651311,None,AxWcLQ==,2023-09-30 21:35:37.944,VIpnnw==,2023-09-30 21:34:36.675,2024-06-24 15:19:32.370296
733,t,potentialVorticity,instant,Temperature,0 days 06:00:00,2.147486e+09,2023-09-30 18:00:00,2023-10-01 00:00:00,gs://global-forecast-system/gfs.20230930/18/at...,536064885,651260,None,AxWcLQ==,2023-09-30 21:35:37.944,VIpnnw==,2023-09-30 21:34:36.675,2024-06-24 15:19:32.370296
734,gh,potentialVorticity,instant,Geopotential height,0 days 06:00:00,2.147486e+09,2023-09-30 18:00:00,2023-10-01 00:00:00,gs://global-forecast-system/gfs.20230930/18/at...,536716145,1175436,None,AxWcLQ==,2023-09-30 21:35:37.944,VIpnnw==,2023-09-30 21:34:36.675,2024-06-24 15:19:32.370296
735,pres,potentialVorticity,instant,Pressure,0 days 06:00:00,2.147486e+09,2023-09-30 18:00:00,2023-10-01 00:00:00,gs://global-forecast-system/gfs.20230930/18/at...,537891581,1135959,None,AxWcLQ==,2023-09-30 21:35:37.944,VIpnnw==,2023-09-30 21:34:36.675,2024-06-24 15:19:32.370296


In [6]:



#gfs_kind

gfs_files = [
    "/home/sparrow/Documents/sewaa_data/gfs.t00z.pgrb2.0p25.f000",
    "/home/sparrow/Documents/sewaa_data/gfs.t00z.pgrb2.0p25.f003"
]

# This operation reads two of the large Grib2 files from GCS
# scan_grib extracts the zarr kerchunk metadata for each individual grib message
# grib_tree builds a zarr/xarray compatible hierarchical view of the dataset
gfs_grib_tree_store = grib_tree([group for f in gfs_files for group in scan_grib(f)])
deflated_gfs_grib_tree_store = copy.deepcopy(gfs_grib_tree_store)
#dynamic_zarr_store.strip_datavar_chunks(deflated_gfs_grib_tree_store)

Dropping unknown variable in msg# 578. Compare with the grib idx file to help identify it and build an ecCodes local grib definitions file to fix it.
Dropping unknown variable in msg# 586. Compare with the grib idx file to help identify it and build an ecCodes local grib definitions file to fix it.
Dropping unknown variable in msg# 665. Compare with the grib idx file to help identify it and build an ecCodes local grib definitions file to fix it.
Dropping unknown variable in msg# 1274. Compare with the grib idx file to help identify it and build an ecCodes local grib definitions file to fix it.
Dropping unknown variable in msg# 1275. Compare with the grib idx file to help identify it and build an ecCodes local grib definitions file to fix it.
Dropping unknown variable in msg# 1285. Compare with the grib idx file to help identify it and build an ecCodes local grib definitions file to fix it.
Dropping unknown variable in msg# 1407. Compare with the grib idx file to help identify it and bu

In [7]:


#%%time
# It is fast to rebuild the datatree - but lets pull out two varables to look at...
gfs_store = dynamic_zarr_store.reinflate_grib_store(
    axes=axes,
    aggregation_type=dynamic_zarr_store.AggregationType.HORIZON,
    #chunk_index=gfs_kind.loc[~gfs_kind.varname.isin(["cape","cin","cpofp","gh","hlcy","mslet","soilw","st","vis"])],
    chunk_index=gfs_kind.loc[gfs_kind.varname.isin(["t2m"])],
    #chunk_index=gfs_kind,
    zarr_ref_store=deflated_gfs_grib_tree_store
)



In [9]:
with open(f'gfs-demo-zstore-dgc-method.pkl', 'wb') as pickle_file:
        pickle.dump(gfs_store, pickle_file)

## loading the data

In [10]:
with open('gfs-demo-zstore-dgc-method.pkl', 'rb') as pickle_file:
    gfs_store = pickle.load(pickle_file)


In [11]:
gfs_dt = datatree.open_datatree(fsspec.filesystem("reference", fo=gfs_store).get_mapper(""), engine="zarr", consolidated=False)
gfs_dt

DataTree('None', parent=None)
└── DataTree('t2m')
    │   Dimensions:  ()
    │   Data variables:
    │       *empty*
    │   Attributes:
    │       name:     2 metre temperature
    └── DataTree('instant')
        │   Dimensions:  ()
        │   Data variables:
        │       *empty*
        │   Attributes:
        │       stepType:  instant
        └── DataTree('heightAboveGround')
                Dimensions:            (latitude: 721, longitude: 1440, model_horizons: 1,
                                        valid_times: 120)
                Coordinates:
                    heightAboveGround  float64 8B ...
                  * latitude           (latitude) float64 6kB 90.0 89.75 89.5 ... -89.75 -90.0
                  * longitude          (longitude) float64 12kB 0.0 0.25 0.5 ... 359.5 359.8
                    step               (model_horizons, valid_times) timedelta64[ns] 960B ...
                    time               (model_horizons, valid_times) datetime64[ns] 960B ...
                    valid_time         (model_horizons, valid_times) datetime64[ns] 960B ...
                Dimensions without coordinates: model_horizons, valid_times
                Data variables:
                    t2m                (model_horizons, valid_times, latitude, longitude) float64 997MB ...
                Attributes:
                    typeOfLevel:  heightAboveGround

In [9]:
with open('zstore-dgc-method-v1.pkl', 'rb') as pickle_file:
    gfs_store_v1 = pickle.load(pickle_file)


In [10]:
gfs_ds = xr.open_dataset(
    fsspec.filesystem("reference", fo=gfs_store_v1).get_mapper(""),
    engine="zarr",
    consolidated=False
)
gfs_ds

<xarray.Dataset> Size: 0B
Dimensions:  ()
Data variables:
    *empty*

In [12]:
aa=gfs_dt["t2m/instant/heightAboveGround"].to_dataset()
#aa1=aa["instant/heightAboveGround"].to_dataset()
aa['time'] = aa.time.astype(str)
aa['valid_time'] = aa.valid_time.astype(str)
#aa['step'] = aa.step.astype(str)
aa1=aa.drop_vars('step')

In [13]:
aa1

<xarray.Dataset> Size: 997MB
Dimensions:            (latitude: 721, longitude: 1440, model_horizons: 1,
                        valid_times: 120)
Coordinates:
    heightAboveGround  float64 8B ...
  * latitude           (latitude) float64 6kB 90.0 89.75 89.5 ... -89.75 -90.0
  * longitude          (longitude) float64 12kB 0.0 0.25 0.5 ... 359.5 359.8
    time               (model_horizons, valid_times) <U48 23kB '2023-09-01T00...
    valid_time         (model_horizons, valid_times) <U48 23kB '2023-09-01T06...
Dimensions without coordinates: model_horizons, valid_times
Data variables:
    t2m                (model_horizons, valid_times, latitude, longitude) float64 997MB ...
Attributes:
    typeOfLevel:  heightAboveGround

In [7]:
import pandas as pd
for date in pd.date_range("2023-09-01", "2023-09-05"):
  for runtime in range(0,63,3):
    horizon=3
    fname=f"gs://global-forecast-system/gfs.{date.strftime('%Y%m%d')}/00/atmos/gfs.t00z.pgrb2.0p25.f{runtime:03}"
    print(fname)

gs://global-forecast-system/gfs.20230901/00/atmos/gfs.t00z.pgrb2.0p25.f000
gs://global-forecast-system/gfs.20230901/00/atmos/gfs.t00z.pgrb2.0p25.f003
gs://global-forecast-system/gfs.20230901/00/atmos/gfs.t00z.pgrb2.0p25.f006
gs://global-forecast-system/gfs.20230901/00/atmos/gfs.t00z.pgrb2.0p25.f009
gs://global-forecast-system/gfs.20230901/00/atmos/gfs.t00z.pgrb2.0p25.f012
gs://global-forecast-system/gfs.20230901/00/atmos/gfs.t00z.pgrb2.0p25.f015
gs://global-forecast-system/gfs.20230901/00/atmos/gfs.t00z.pgrb2.0p25.f018
gs://global-forecast-system/gfs.20230901/00/atmos/gfs.t00z.pgrb2.0p25.f021
gs://global-forecast-system/gfs.20230901/00/atmos/gfs.t00z.pgrb2.0p25.f024
gs://global-forecast-system/gfs.20230901/00/atmos/gfs.t00z.pgrb2.0p25.f027
gs://global-forecast-system/gfs.20230901/00/atmos/gfs.t00z.pgrb2.0p25.f030
gs://global-forecast-system/gfs.20230901/00/atmos/gfs.t00z.pgrb2.0p25.f033
gs://global-forecast-system/gfs.20230901/00/atmos/gfs.t00z.pgrb2.0p25.f036
gs://global-forecast-syst